In [1]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
import numpy as np

In [3]:
def sparseidx(urls):
    title=('CIK', 'Company Name', 'Form Type', 'Date Filed', 'Filename')
    files=pd.DataFrame(columns=title)
    for url in urls:
        lines = requests.get(url).content.decode("utf-8", "ignore").splitlines()
        records = [tuple(line.split('|')) for line in lines[11:]]
        files=pd.concat([files,pd.DataFrame(data=records,columns=title)],axis=0)
        files=files[files['Form Type'].str.contains('13F-HR')]
    files=files.reset_index(drop=True)
    files['Filename']='https://www.sec.gov/Archives/'+files['Filename']
    return files

In [6]:
def extract13F(fileurl):
    response=requests.get(fileurl)
    contentTmp = response.content.decode('utf-8')
    soup=BeautifulSoup(contentTmp,'xml')
    rows=soup.find_all(re.compile('.*infotable.*',re.IGNORECASE))
    colname=['IssuerName','ClassTitle','Cusip','Value','SHRSorPRNAmt','SHorPRN','PUTorCall','IvtDiscretion','OtherManager','Sole','Shared','None']
    itable=[]
    for r in rows:
        texts=list(filter(lambda x :x!='',r.get_text('?').replace('\n','').split('?')))
        if r.find(re.compile(r'putcall',re.IGNORECASE))==None:
            texts.insert(6,None)
        elif r.find(re.compile(r'putcall',re.IGNORECASE)).get_text().replace('\n','')=='':
            texts.insert(6,None)
        if r.find(re.compile(r'othermanager',re.IGNORECASE))==None:
            texts.insert(8,None)
        elif r.find(re.compile(r'othermanager',re.IGNORECASE)).get_text().replace('\n','')=='':
            texts.insert(8,None)
        itable.append(texts)
    table=pd.DataFrame(itable,columns=colname)
    return table

In [5]:
#test
years=[2017,2018,2019]
quaters=[1,2,3,4]
urls=[]
for y in years:
    for q in quaters:
        urls.append('https://www.sec.gov/Archives/edgar/full-index/{0}/QTR{1}/master.idx'.format(y,q))
files=sparseidx(urls)

In [7]:
tfiles=files.iloc[100:200].to_dict('index')
for key in tfiles:
    fileurl=tfiles[key]['Filename']
    table=extract13F(fileurl)
    tfiles[key]['infotable']=table.to_dict('index')
    print(key)
    


100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [9]:
tfiles[100]

{'CIK': '1020317',
 'Company Name': 'PEKIN SINGER STRAUSS ASSET MANAGEMENT /IL',
 'Form Type': '13F-HR/A',
 'Date Filed': '2017-03-24',
 'Filename': 'https://www.sec.gov/Archives/edgar/data/1020317/0001085146-17-000876.txt',
 'infotable': {0: {'IssuerName': '3M CO',
   'ClassTitle': 'COM',
   'Cusip': '88579Y101',
   'Value': '369',
   'SHRSorPRNAmt': '2068',
   'SHorPRN': 'SH',
   'PUTorCall': None,
   'IvtDiscretion': 'SOLE',
   'OtherManager': None,
   'Sole': '0',
   'Shared': '0',
   'None': '2068'},
  1: {'IssuerName': 'ABBOTT LABS',
   'ClassTitle': 'COM',
   'Cusip': '002824100',
   'Value': '843',
   'SHRSorPRNAmt': '21950',
   'SHorPRN': 'SH',
   'PUTorCall': None,
   'IvtDiscretion': 'SOLE',
   'OtherManager': None,
   'Sole': '0',
   'Shared': '0',
   'None': '21950'},
  2: {'IssuerName': 'ABBVIE INC',
   'ClassTitle': 'COM',
   'Cusip': '00287Y109',
   'Value': '3408',
   'SHRSorPRNAmt': '54430',
   'SHorPRN': 'SH',
   'PUTorCall': None,
   'IvtDiscretion': 'SOLE',
   'Oth